In [1]:
from det2Clo import clo2Detector as c2d
from det2Clo import clo2Trainer as c2t
from det2Clo import clo2Imagine as c2i

from os.path import join
from pprint import pprint as pp

In [2]:
# ! Define the paths for clothing
DATA_PATH = '/home/denny/datasets/deepfashion/'
CONF_PATH = join(DATA_PATH, 'conf/train_config.json')
YAML_NAME = 'COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml'
MODL_PATH = join(DATA_PATH, 'outputs/model_0054999.pth')
CATE_PATH = join(DATA_PATH, 'train_descriptions.json')

PATH_FOR_CLOTHING = {
    'DATA_PATH':str(DATA_PATH), 
    'CONF_PATH':str(CONF_PATH), 
    'YAML_NAME':str(YAML_NAME), 
    'MODL_PATH':str(MODL_PATH), 
    'CATE_PATH':str(CATE_PATH)
}

In [3]:
# ! Define the paths for features
DATA_PATH = '/home/denny/datasets/iMaterialist/'
CONF_PATH = join(DATA_PATH, 'conf/train_config.json')
YAML_NAME = 'COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml'
MODL_PATH = join(DATA_PATH, 'outputs/model_0034999.pth')
CATE_PATH = join(DATA_PATH, 'train_categories.json')

PATH_FOR_FEATURES = {
    'DATA_PATH':str(DATA_PATH), 
    'CONF_PATH':str(CONF_PATH), 
    'YAML_NAME':str(YAML_NAME), 
    'MODL_PATH':str(MODL_PATH), 
    'CATE_PATH':str(CATE_PATH)
}

In [4]:
ovelap_hierachy = { v:i for i,v in enumerate([
    'shortsleevetop', 'longsleevetop', 'sling', 'vest',
    'shortsleeveoutwear', 'longsleeveoutwear', 'shortsleevedress', 'longsleevedress', 'slingdress', 'vestdress',
    'trousers', 'shorts', 'skirt',
])}
PATHS = [PATH_FOR_CLOTHING, PATH_FOR_FEATURES]
threshold=[0.3, 0.1]

In [5]:
managers, detectors = list(), list()
for i in range(len(PATHS)) : 
    # ! Make manager for get configuration
    managers.append(c2t.Manager(PATHS[i]['DATA_PATH'], PATHS[i]['YAML_NAME'], PATHS[i]['CONF_PATH']))

    # ! Make detector for detect clothings
    detectors.append(c2d.Detector(managers[i], PATHS[i]['MODL_PATH'], PATHS[i]['CATE_PATH'], threshold=threshold[i], batchsize=512, name=PATHS[i]['DATA_PATH'][-6:]))

In [6]:
# First, we detect the clothing
img_path = join(PATHS[0]['DATA_PATH'], 'sample3.jpg')
clothing_img, outputs = detectors[0].detect(img_path, resize_width=-1, spec=True)
parsed_outputs = detectors[0].parse4specs(outputs)

In [7]:
# ! Make stage for show the detected image
stage = c2i.Stage()
stage.show(clothing_img, tool="cv2", scale=0.8, detects=outputs)

In [8]:
# Next, we detect the features from masked images
masked_imgs = detectors[0].mask2img(clothing_img, parsed_outputs, overlapping=True)
feature_img, outputs = detectors[1].detect(masked_imgs[0], resize_width=-1, spec=True)

In [9]:
feature_img, outputs = detectors[1].detect(masked_imgs[1], resize_width=-1, spec=True)
stage.show(feature_img, tool="cv2", scale=0.8, detects=outputs)

In [10]:
show = c2i.Show(stage)

In [11]:
from cv2  import cv2
from time import sleep
import numpy as np

In [16]:
def show_detected_clothing_to_camera() :
    # ! Camera modules
    sleep_time_ms = 10

    window_main = ['VideoFrame']
    window_list = ['Segmentation_Clothing_1', 'Segmentation_Clothing_2', 'Segmentation_Clothing_3']
    for wname in (window_main+window_list) : cv2.namedWindow(wname, cv2.WINDOW_NORMAL)
    capture = cv2.VideoCapture(0)
    capture.set(cv2.CAP_PROP_FRAME_WIDTH,  640)
    capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    

    try :
        while True :
            ret, frame = capture.read()
            if ret : 
                cv2.imshow(window_main[0], frame)

                img, outputs = detectors[0].detect(frame, spec=True)
                parsed_outputs = detectors[0].parse4specs(outputs, hierachy=ovelap_hierachy)
                masked_imgs = detectors[0].mask2img(img, parsed_outputs, overlapping=True)

                for i, mimg in enumerate(masked_imgs[:len(window_list)]) :
                    # Put some text to images
                    cv2.putText(
                        img=mimg, 
                        text="Category name : {}, number : {}".format(parsed_outputs['labels'][i], parsed_outputs['classes'][i]), 
                        org=(50, 50),
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                        fontScale=0.7,
                        color=(0,0,0),
                        thickness=2,
                        lineType=cv2.LINE_AA,
                    )
                    
                    fimg, outputs = detectors[1].detect(mimg, spec=True)
                    
                    # Show images
                    cv2.imshow(window_list[i], stage.paint(fimg, outputs, scale=1))
                    
                # default screen
                for i in range(0, len(window_list)-len(masked_imgs), 1) :
                    cv2.imshow(window_list[len(window_list)-i-1], np.zeros(shape=(100, 100, 3)))

            if cv2.waitKey(sleep_time_ms) > 0 :break
    except Exception as e :
        print("ERROR :", str(e))
    finally : 
        print("Done.")
        capture.release()
        cv2.destroyAllWindows()

In [17]:
show_detected_clothing_to_camera()

Done.


In [ ]:
# stage.draw(stage.paint(img, outputs))